In [28]:
!pip install ffmpeg-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 10.7 MB/s eta 0:00:00 MB/s eta 0:00:01


In [1]:
import os
import wave
import pyaudio
from pydub import AudioSegment

In [23]:
def play_wav_file(wav_file_path):
    with wave.open(wav_file_path, 'rb') as wf:
        p = pyaudio.PyAudio()
        
        print(f"sample width: {wf.getsampwidth()}")
        print(f"n channels: {wf.getnchannels()}")
        print(f"framerate: {wf.getframerate()}")
        
        stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                        channels=wf.getnchannels(),
                        rate=wf.getframerate(),
                        output=True)

        # Start the stream
        stream.start_stream()

        CHUNK = 1024
        data = wf.readframes(CHUNK)

        print(f"Playing...from {wav_file_path}")
        while data:
            stream.write(data)
            data = wf.readframes(CHUNK)

        # Stop and close the stream
        stream.stop_stream()
        stream.close()
        p.terminate()

        print("finished playing")


In [14]:
webm_file_path = os.path.join('recordings', 'recorded_audio.webm')
wav_file_path = os.path.join('recordings', 'recorded_audio.wav')
mp4_file_path = os.path.join('recordings', 'recorded_audio.mp4')
wav_file_path2 = os.path.join('recordings', 'output_48000.wav')

In [9]:
# Convert the audio to WAV format (you can use pydub or similar)
# sound = AudioSegment.from_file(webm_file_path, format="webm")
# sound.export(wav_file_path, format="wav")

<_io.BufferedRandom name='recordings/recorded_audio.wav'>

In [24]:
play_wav_file(wav_file_path)

ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map


sample width: 4
n channels: 1
framerate: 48000
Playing...from recordings/recorded_audio.wav
finished playing


In [26]:
play_wav_file(wav_file_path2)

ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map


sample width: 2
n channels: 1
framerate: 48000
Playing...from recordings/output_48000.wav


ALSA lib pcm.c:8306:(snd_pcm_recover) underrun occurred


finished playing


In [8]:
import vlc

def play_webm(file_path, device_name=None):
    # Initialize VLC instance
    instance = vlc.Instance()

    # Create a media player
    player = instance.media_player_new()

    # Set media
    media = instance.media_new(file_path)
    player.set_media(media)

    # Set the desired audio output device if specified
    if device_name:
        # Get the audio output module name (e.g., 'alsa', 'coreaudio', etc.)
        audio_output_module = player.audio_output_device_get()
        
        # Set the specific audio output device
        player.audio_output_device_set(audio_output_module, device_name)
    
    # Play the media
    player.play()
    
    # Keep the program running until the audio finishes
    while player.is_playing():
        pass


[00007f1908001830] filesystem stream error: cannot open file /home/congyu/congyu_program/pythons/inits/p_toy_car_controller/test_hardware/path/to/your/file.webm (No such file or directory)
[0000559d5b921950] main input error: Your input can't be opened
[0000559d5b921950] main input error: VLC is unable to open the MRL 'file:///home/congyu/congyu_program/pythons/inits/p_toy_car_controller/test_hardware/path/to/your/file.webm'. Check the log for details.


In [9]:
# Example usage: play a file on a specific device
play_webm(webm_file_path)

In [12]:
sound = AudioSegment.from_file(webm_file_path, format="webm")
sound.export(mp4_file_path, format="mp4")

<_io.BufferedRandom name='recordings/recorded_audio.mp4'>

In [25]:
import subprocess
import json

def get_webm_info(webm_file_path):
    cmd = [
        'ffprobe', '-v', 'error', '-show_entries',
        'format=duration:stream=codec_name,codec_type,width,height,r_frame_rate,sample_rate',
        '-of', 'json', webm_file_path
    ]
    
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    info = json.loads(result.stdout)
    
    video_frame_rate = None
    audio_sample_rate = None

    for stream in info['streams']:
        if stream['codec_type'] == 'video':
            r_frame_rate = stream['r_frame_rate']
            num, denom = map(int, r_frame_rate.split('/'))
            video_frame_rate = num / denom
        elif stream['codec_type'] == 'audio':
            audio_sample_rate = int(stream['sample_rate'])

    return video_frame_rate, audio_sample_rate

# Example usage
video_fps, audio_sample_rate = get_webm_info(webm_file_path)
print(f"Video Frame Rate: {video_fps} FPS")
print(f"Audio Sample Rate: {audio_sample_rate} Hz")


Video Frame Rate: None FPS
Audio Sample Rate: 48000 Hz


In [32]:
import pyaudio
import ffmpeg
import numpy as np

def play_webm_from_binary(webm_binary_data):
    # Use FFmpeg to decode the binary data of the webm file to raw PCM data
    process = (
        ffmpeg
        .input('pipe:0', format='webm')
        .output('pipe:', format='wav')
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True)
    )

    # Feed the binary data to ffmpeg's stdin
    process.stdin.write(webm_binary_data)
    process.stdin.close()

    # Initialize PyAudio
    p = pyaudio.PyAudio()

    # Open a stream with PyAudio for playback
    stream = p.open(format=pyaudio.paInt16,
                    channels=2,
                    rate=48000,
                    output=True)

    chunk_size = 1024

    while True:
        data = process.stdout.read(chunk_size)
        if not data:
            break
        stream.write(data)

    # Stop and close the stream
    stream.stop_stream()
    stream.close()
    p.terminate()

    process.stdout.close()
    process.wait()

# Example usage
# Assuming you have a binary representation of the webm file
with open(webm_file_path, 'rb') as f:
    webm_binary = f.read()

play_webm_from_binary(webm_binary)


ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map


In [34]:
import pyaudio
import ffmpeg
import numpy as np

def play_webm_from_binary(webm_binary_data):
    # Use FFmpeg to decode the binary data of the webm file to raw PCM data
    process = (
        ffmpeg
        .input('pipe:0', format='webm')
        .output('pipe:', format='wav')
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True)
    )

    # Feed the binary data to ffmpeg's stdin
    process.stdin.write(webm_binary_data)
    process.stdin.close()

    # Read the first few bytes of the WAV header to determine the sample rate and number of channels
    wav_header = process.stdout.read(44)  # WAV header is 44 bytes
    channels = int.from_bytes(wav_header[22:24], byteorder='little')
    sample_rate = int.from_bytes(wav_header[24:28], byteorder='little')

    print(f"Channels: {channels}, Sample Rate: {sample_rate}")

    # Initialize PyAudio
    p = pyaudio.PyAudio()

    # Open a stream with PyAudio for playback
    stream = p.open(format=pyaudio.paInt16,
                    channels=channels,
                    rate=sample_rate,
                    output_device_index=6,
                    output=True)

    chunk_size = 1024

    while True:
        data = process.stdout.read(chunk_size)
        if not data:
            break
        stream.write(data)

    # Stop and close the stream
    stream.stop_stream()
    stream.close()
    p.terminate()

    process.stdout.close()
    process.wait()

# Example usage
# Assuming you have a binary representation of the webm file
with open(webm_file_path, 'rb') as f:
    webm_binary = f.read()

play_webm_from_binary(webm_binary)


Channels: 1, Sample Rate: 48000


ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:867:(find_matching_chmap) Found no matching channel map
